# Setup

This notebook illustrates how to access the Submodular problems using IOHexperimenter.
To install IOHexperimenter, the following command can be used:

In [ ]:
#%pip install ioh

Once installed, simply import the package

In [ ]:
import ioh

# Accessing problems

In ioh, everything revolves around the problem class, so we start by creating an individual submodular problem.
To see which problems are available, we can check:

In [ ]:
ioh.ProblemClass.GRAPH.problems

We can then construct a problem as follows:

In [ ]:
f = ioh.get_problem(2000, problem_class=ioh.ProblemClass.GRAPH)

This problem contains all kinds of information about the way it is constructed. It also keeps track of all evaluations, which can be seen as follows:

In [ ]:
print(f.meta_data)
print(f.state)

Since this problem can be evaluated as a common python function, it should work directly with any optimizer. We make a basic random search to illustrate this:

In [ ]:
import numpy as np

class RandomSearch:
    'Simple random search algorithm'
    def __init__(self, n: int, length: float = 0.0):
        self.n: int = n
        
    def __call__(self, problem: ioh.problem.IntegerSingleObjective) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        
        for _ in range(self.n):
            # We can use the problems meta information to see the number of variables needed
            x = np.random.randint(0, 2, size=problem.meta_data.n_variables)
            problem(x)   

If we want to run this algorithm on our problem, we can do the following:

In [ ]:
r = RandomSearch(10)
r(f)
print(f.state)

To run multiple independent runs on the same problem, we can reset the state as follows:


In [ ]:
f.reset()

# Logging

The default usage of IOHExperimenter is in generating logs of benchmarking experiments which can be analyzed in IOHAnalyzer.

In [ ]:
import os

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="my-experiment",       # in a folder named: 'my-experiment'
    algorithm_name="random-search",    # meta-data for the algorithm used to generate these results
    store_positions=False               # disable storing x-variables in the logged files
)

# this automatically creates a folder 'my-experiment' in the current working directory
# if the folder already exists, it will given an additional number to make the name unique
logger

We can add this logger to a problem so we can store the data when running our algorithm

In [ ]:
f.attach_logger(logger)

In [ ]:
r = RandomSearch(100)
r(f)

Once finished with the run, we can close the logger to force it to write the data (happens automatically when running as a python script)

In [ ]:
logger.close()

# Experiment class

In Python, we provide the Experiment class which can be used to easily run a given algorithm over a larger number of problems.

In [ ]:
experiment = ioh.Experiment(
    algorithm = RandomSearch(10), # An algorithm instance
    fids = list(ioh.ProblemClass.GRAPH.problems.keys())[:3],               # the id's of the problems we want to test
    reps = 3,                     # the number of runs,
    problem_class = ioh.ProblemClass.GRAPH, #the problem type
    zip_output = True       
)

Running this experiment creates a zip-file, which can directly be processed by IOHanalyzer (https://iohanalyzer.liacs.nl)

In [ ]:
experiment()

For more background and other information on using IOHexperimenter, we refer to https://iohprofiler.github.io/IOHexp/

If you have any questions about the usage of the submodular problems, or about IOHprofiler in general, please make an issue on https://github.com/IOHprofiler/IOHexperimenter

In [ ]:
# cleanup
from shutil import rmtree
import os
import glob

def clean():
    for name in ("my-experiment", "ioh_data"):
        for path in glob.glob(f"{name}*"):
            if os.path.isfile(path):
                os.remove(path)
            if os.path.isdir(path):
                rmtree(path, ignore_errors=True)

def ls(p="./"):
    for obj in os.listdir(os.path.normpath(p)):
        print(obj)

def cat(f):
    with open(os.path.normpath(f)) as h:
        print(h.read())

clean()

rmtree("my-experiment", ignore_errors=True)
rmtree("ioh_data", ignore_errors=True)